# MLCV Geospatial Analysis - R Code

In [ ]:
# Importing Necessary Libraries
library(tidyverse)
library(zipcode)
library(maps)
library(maptools)
library(RColorBrewer)

In [ ]:
# Reading in Required Shapefile of U.S. Census Bureau Zip Code Tabulation Areas
map_shp <- readShapePoly("D://Group Folder/MLCV_Zip/MLCV_Zip/mn_zip_joined.shp")
# from https://catalog.data.gov/; initial data preprocessing performed using QGIS to
# filter data to include State: 027.

A Note about Zip Code Tabulation Areas (ZCTAs): A generalized representations of US Postal Service Zip Code service areas, which have been revised to better align with the geographic boundaries used by the U.S. Census Bureau. Due to the resulting approximations, the results of our analysis should, in turn, be treated as an approximation. For more information, please refer to: census.gov/programs-surveys/geography/guidance/geo-areas/zctas.html. 

In [ ]:
# Reviewing first 5 rows of data within the shapefile
head(map_shp@data)

In [ ]:
# Plotting map of Minnesota ZCTAs
plot(map_shp)

In [ ]:
# Importing Tabular Data

# Extracting national-level data from zipcode library
data(zipcode) 
# Table of player characteristics
dim_player <- read.csv("D://Group Folder/Datasets/DimPlayer.csv") 
# Isolating and counting zip codes
zip_data <- dim_player %>% count(ZipCode) %>% arrange(desc(n))
# Cleaning zip codes using the zipcode library
zip_data$ZipCode <- zipcode::clean.zipcodes(zip_data$ZipCode)
# Joining national-level data to zip code counts
df <- left_join(zip_data, zipcode, by=c("ZipCode"="zip"))
# Importing American Community Survey (ACS) 5-Year Estimates; 
# Obtained from https://census.gov/programs-surveys/acs/
acs_data <- read.csv(paste0("D://Group Folder/MLCV_Zip/MLCV_Zip/",
                            "ACS_17_5YR_DP05_with_ann.csv"), skip = 1)[1:5]
# Changing dataframe names
names(acs_data) <- c("Id", "Id2", "Geography", "Estimate", "Margin")
# Ensuring IDs are interpreted as char rather than numeric
acs_data$Id2 <- as.character(acs_data$Id2)
# Joining ACS Data to zipcode/count dataframe
all_data <- left_join(df, acs_data, by=c("ZipCode"="Id2"))
# (Important Step) Standardizing zip code counts
all_data <- all_data %>% mutate(Percentage = n / Estimate)
# Safeguarding against duplicates resulting from joins
all_data <- all_data[complete.cases(all_data),]
# Eliminating Null/NA Joins
all_data <- all_data %>% filter(n > 1)
# Ensuring GEOIDs are interpreted as char rather than numeric
map_shp@data$GEOID10 <- as.character(map_shp@data$GEOID10)
# merging tabulated data to the shapefile for mapping
map_shp2 <- merge(map_shp, all_data, by.x="GEOID10", by.y="ZipCode")

In [ ]:
# Preliminary data analysis; zip codes per county
all_data_20 <- all_data %>% filter(Estimate > 0) %>% arrange(desc(Percentage)) %>% top_n(20)
ggplot(all_data_20 , aes(x=reorder(city, Percentage), y=Percentage)) + geom_col() + coord_flip()
ggplot(all_data_20 , aes(x=reorder(ZipCode, Percentage), y=Percentage)) + geom_col() + coord_flip()

**Plotting zip codes with standardization**

In [ ]:
# Creating a color palette and assigning weights 
p <- colorRampPalette((c("white", "red")))(100)
palette(p)
pop <- map_shp2@data$Percentage * 200

# plotting standardized zip code counts (Important Figure)
plot(map_shp, col = pop)
# adding titles
title(main="Scaled ZipCode Densities")

**Plotting national zip codes without standardization**

In [ ]:
# Preparing necessary the underlying maps
county <- map_data("county", region = "minnesota")
mn <- map_data("state", region = "minnesota")
us <- map_data("state")

In [ ]:
# Nation-wide plot of zip code densities
ggplot(us) + 
  geom_polygon( aes(x=long, y=lat, group = group), fill="white", color="black") +
  coord_quickmap() +
  stat_density2d(aes(x=longitude, y=latitude, fill=..level.., alpha=..level..), 
                 bins=9, geom="polygon", data=df) +
  scale_fill_gradient(low="black", high="red") +
  theme_bw() + theme(legend.position = 'none') + 
  labs(title = "Mille Lacs Casino\n", subtitle = "Customer Zip Code Density")

In [ ]:
# Plotting nation-wide zip codes with density gradients
  geom_polygon( aes(x=long, y=lat, group = group), fill="white", color="black") +
  coord_quickmap() +
  geom_point(aes(x=longitude, y=latitude, color=n), size = 0.2, data=df) +
  scale_x_continuous(limits = c(-125, -66)) + 
  scale_y_continuous(limits = c(25, 50)) +
  scale_color_viridis_c() +
  scale_fill_gradient(low="black", high="red") +
  theme_classic() + 
  labs(title = "Mille Lacs Casino\n", subtitle = "Customer Zip Code Density")

**Plotting Minnesota zip codes without standardization**

In [ ]:
# Plotting state-wide zip codes with density gradients
df_mn <- df %>% filter(state=="MN")
ggplot(county) + 
  geom_polygon(aes(x=long, y=lat, group=group), fill="white", color="black") +
  coord_quickmap() +
  geom_point(aes(x=longitude, y=latitude, color=n), size=1, data=df_mn) +
  scale_x_continuous(limits = c(-97.5, -89)) + 
  scale_y_continuous(limits = c(43, 50)) +
  scale_color_viridis_c(option="magma") +
  scale_fill_gradient(low="black", high="red") +
  theme_bw()

In [ ]:
# Plotting state-wide zip code densities with reference locations
mpls <- data.frame(longitude=c(-93.2650), latitude=c(44.9778), label=c("MPLS"))
mlc <-  data.frame(longitude=c(-93.7591), latitude=c(46.1799), label=c("MLC"))
df_mn <- df %>% filter(state=="MN")

# Mapping Zip Codes
ggplot(county) + 
  geom_polygon(aes(x=long, y=lat, group=group), fill="white", color="black") +
  coord_quickmap() +
  geom_point(aes(x=longitude, y=latitude, color=n), size=1, data=df_mn) +
  scale_x_continuous(limits = c(-97.5, -89)) + 
  scale_y_continuous(limits = c(43, 50)) +
  scale_color_viridis_c(option="magma") +
  scale_fill_gradient(low="black", high="red") +
  stat_density2d(aes(x=longitude, y=latitude, fill=..level.., alpha=..level..),
                 bins=9, geom="polygon", data=df_mn) +
  geom_point(aes(x=longitude, y=latitude), size=3, fill="blue", shape=23, data=mpls) +
  geom_text(data=mpls, aes(x=longitude, y=latitude), color="blue", size=5,
            label="Minneapolis", hjust=-.3) +
  geom_point(aes(x=longitude, y=latitude), size=3, fill="blue", shape=23, data=mlc) +
  geom_text(data=mlc, aes(x=longitude, y=latitude), color="blue", size=5,
            label="Mille Lacs Casino", hjust=-.1) +
  theme_bw() + theme(legend.position = 'none') + 
  labs(title = "Mille Lacs Casino\n", subtitle = "Customer Zip Code Density")

**State-level Descriptive Statistics**

In [ ]:
# Zip code counts by city
df_temp <- df %>% 
  group_by(city) %>% 
  summarise(total_n = sum(n)) %>% 
  arrange(desc(total_n)) %>% 
  top_n(40) %>% filter(!is.na(city))

ggplot(df_temp, aes(x=reorder(city, total_n), y=total_n)) + 
  geom_col() + coord_flip() + 
  geom_text(label=scales::comma(df_temp$total_n), hjust=-.4) +
  scale_y_continuous(limits = c(0, 1.1* max(df_temp$total_n)), labels = scales::comma) +
  labs(title="Mille Lacs Casino\n", subtitle="Zip Code Counts", x="", y="Count") + 
  theme_bw()